**Update the dynamic of the model**  
2020-6-13

- In the previous version, the B_ and D_ was fixed, and reallocate evenly
- may research on the diffrent settings 
- In this version, allocate the bikes according to the arrival ratex
- In the next set the D_ can be the bikes left in the distribute pool

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sympy as sp
from math import factorial
from math import isclose
from tqdm import tqdm
import csv
from scipy.sparse import csr_matrix
from scipy.sparse import identity
from scipy.sparse.linalg import spsolve
from numba import jit
from numpy.random import choice

In [17]:
# set the parameters
# get the number of states S
'''
A: number of areas
M: number of total bikes
S: number of total states
Pij: transfering possibility matrix
Beta: broken rate
ArrMtx: arrival rates of each area
Gamma: gathering rate
Mu: fix rate
Delta: distributing rate
RhoMtx: matrix of ride rates
N: number of fix servers
B_: valve value at broken pool
D_: valve value at distributing pool
'''

A = 2
M = 6
Pij = [[0.3, 0.7],
       [0.7, 0.3]]
# Pij = [[0.1, 0.2, 0.3, 0.4],
#        [0.2, 0.3, 0.4, 0.1],
#        [0.3, 0.4, 0.1, 0.2],
#        [0.4, 0.3, 0.2, 0.1]]
Beta = 0.4
ArrLst = [2.5, 5.0]
#ArrLst = [5.0, 5.0, 6.0, 7.0]
Gamma = 1.0
Mu = 1.0
Delta = 1.0
# RhoMtx = [[1.0, 1.0], 
#           [1.0, 1.0]]
RhoMtx = [[1.0] * A for i in range(A)]
N = 4
B_, D_ = 3, 3

In [18]:
# generate state dictionary direrctly

def temp(x, i, index, possSet):
    x.append(i)
    generate_R(x.copy(), index, possSet)

def generate_R(s, index, possSet):
    L = A+3+A**2
    if len(s)==L and sum(s)==M and sum(s[-2:]) in possSet:
        State[tuple(s)] = index[0]
        index[0] += 1
    elif len(s)>L or sum(s)>M or (len(s)==L and not sum(s[-2:]) in possSet):
        return 0
    else:
        for i in range(M+1):
            temp(s.copy(), i, index, possSet)
def getPossSet():
    possSet = []
    inLst = [i for i in range(0, M+1, B_)]
    #print(inLst)
    for _ in inLst:
        fake = [i for i in range(_, -1, -D_)]
        possSet += fake
        #print(fake)
    return set(possSet)
def getState():
    # generate state dictionary direrctly
    State = {}
    index = [0]
    possSet = getPossSet()
    for i in range(M+1):
        generate_R([i], index, possSet)
    return State

State = {}
%time getState()

n_state = len(State)
n_queue = A+3+A**2
print(n_queue, n_state)

CPU times: user 67.3 ms, sys: 2.42 ms, total: 69.7 ms
Wall time: 69.5 ms
9 1267


In [19]:
def get_target_number():
    #A, M, D_, ArrLst = A, M, D_, ArrLst
    arr = np.array(ArrLst)/sum(ArrLst)
    num_dis = [int(M*x) for x in arr]
    left = M - sum(num_dis)
    for i in np.argsort(ArrLst)[::-1][:left]:
        num_dis[i] += 1
    return num_dis
num_dis = get_target_number()
num_dis

[2, 4]

In [6]:
n, col, val = 0, [], []
tempS = [3,3,0,0,0]
arr_rank = np.argsort(num_dis)[::-1]
N_lst = tempS[:A]
dbar = D_
#print(arr_rank, num_dis,N_lst1, dbar1)
result_lst = []
#    A, D_, num_dis = A, D_, num_dis
# DP, Ni
def get_before_reallocate(arr_rank, num_dis, N_lst, dbar, result_lst):
    #if dbar == 0: return result_lst
    #print(arr_rank, num_dis, N_lst, dbar, result_lst)
    i = arr_rank[0]
    if N_lst[i]<num_dis[i]:
        if dbar<=N_lst[i]:
            result = N_lst.copy()
            result[i] -= dbar
            result_lst.append(result)
    elif N_lst[i]>num_dis[i]:
        if arr_rank.size!=0:
            get_before_reallocate(arr_rank[1:], num_dis, N_lst, dbar, result_lst)
    else:
        if arr_rank.size==1:
            if N_lst[i]>=dbar: 
                result = N_lst.copy()
                result[i] -= dbar
                result_lst.append(result)
        else: 
            for j in range(min(N_lst[i], dbar)+1):
                arr_rank1 = arr_rank[1:]
                result = N_lst.copy()
                result[i] -= j
                get_before_reallocate(arr_rank1, num_dis, result, dbar-j, result_lst)
    
for s in State:
    tempS = list(s)
    #    A, D_, num_dis = A, D_, num_dis
        # DP, Ni
    arr_rank = np.argsort(num_dis)[::-1]
    N_lst1 = tempS[:A]
    tempsum =sum(N_lst1)
    dbar1 = D_
    result_lst = []
    get_before_reallocate(arr_rank, num_dis, N_lst1, dbar1, result_lst)
    result_lst1 = [tuple(x) for x in result_lst]
    assert(len(result_lst)==len(set(result_lst1)))
        
#     tempS[-1] += D_
#     tail = tempS[-3:]
#     for x in result_lst:
#         temp = x+tail
#         assert(len(result_lst)==len(set(result_lst)))
#         if sum(x)-tempsum!=-D_:
#             print(x, tempS)
        #assert(sum(N_lst1)-sum(temp)==D_)

In [20]:
# 使用scipy.sparse.csr_sparse
# generate R matrix

def INi(ni):
    if ni > 0: return 1
    else: return 0
def IBP(bp):
    if bp >= B_: return 1
    else: return 0
def IDP(dp):
    if dp >= D_: return 1
    else: return 0
def IS(s):
    # 判断是不是都是在0到M之间
    if tuple(s) in State.keys(): return 1
    else: return 0
    
def get_target_number():
    #A, M, D_, ArrLst = A, M, D_, ArrLst
    arr = np.array(ArrLst)/sum(ArrLst)
    num_dis = [int(M*x) for x in arr]
    left = M - sum(num_dis)
    for i in np.argsort(ArrLst)[::-1][:left]:
        num_dis[i] += 1
    return num_dis
num_dis = get_target_number()

def arrRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            # Ni, Rij
            y = A + i*A + j
            a1 = tempS[i] = tempS[i] + 1
            a2 = tempS[y] = tempS[y] - 1
            if IS(tempS):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(ArrLst[i]*Pij[i][j])
    #         print(tempS)
    # print(n, col, val)
    # print('arr-----------')
    return n, col, val
            
def backRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            # Ni, Rij
            y = A + j*A + i
            a1 = tempS[i] = tempS[i] - 1
            a2 = tempS[y] = tempS[y] + 1
            if IS(tempS):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(RhoMtx[j][i]*a2*(1-Beta))
    #         print(tempS)
    # print(n, col, val)
    # print('back-----------')
    return n, col, val

def broRateIn(s):
    n, col, val = 0, [], []
    for i in range(A):
        for j in range(A):
            tempS = list(s)
            #Rij, BP
            y = A + i*A + j
            a1 = tempS[y] = tempS[y] + 1
            a2 = tempS[-3] = tempS[-3] - 1
            if IS(tempS):
                n += 1
                col.append(State[tuple(tempS)])
                val.append(RhoMtx[i][j]*a1*Beta)
    #         print(tempS)
    # print(n, col, val)
    # print('bro-----------')
    return n, col, val

def gathRateIn(s):
    n, col, val = 0, [], []
    tempS = list(s)
    # BP, RC
    a1 = tempS[-3] = tempS[-3] + B_
    a2 = tempS[-2] = tempS[-2] - B_
    if IS(tempS):
        n += 1
        col.append(State[tuple(tempS)])
        val.append(Gamma)
    # print(tempS)
    # print(n, col, val)
    # print('gath-----------')
    return n, col, val

def fixRateIn(s):
    n, col, val = 0, [], []
    tempS = list(s)
    # RC, DP
    a1 = tempS[-2] = tempS[-2] + 1
    a2 = tempS[-1] = tempS[-1] - 1
    if IS(tempS):
        n += 1
        col.append(State[tuple(tempS)])
        val.append(Mu*min(a1, N))
    # print(tempS)
    # print(n, col, val)
    # print('fix-----------')
    return n, col, val

def redRateIn(s):
    n, col, val = 0, [], []
    tempS = list(s)
#    A, D_, num_dis = A, D_, num_dis
    # DP, Ni
    arr_rank, N_lst1, dbar1, result_lst = np.argsort(num_dis)[::-1], tempS[:A], D_, []
    get_before_reallocate(arr_rank, num_dis, N_lst1, dbar1, result_lst)
    tempS[-1] += D_
    tail = tempS[A:]
    for x in result_lst:
        temp = x+tail
        if IS(temp):
            n += 1
            col.append(State[tuple(temp)])
            val.append(Delta)
    # print(tempS)
    # print(n, col, val)
    # print('red-----------')
    return n, col, val
    
def getRateOut(s):
    outRate = 0
    for i in range(A):
        if INi(s[i]):
            outRate += ArrLst[i]
        else: continue
    for i in range(A):
        for j in range(A):
            outRate += RhoMtx[i][j] * s[A+i*A+j]
    outRate += Gamma*IBP(s[-3]) + Mu*min(s[-2], N) + Delta*IDP(s[-1])
    # inRate += sum(list(map(lambda a: a[0]*a[1], zip(x,y))))
    return -outRate

def getRateIn(s):
    n, col, val = 0, [], []
    '''
    # customer arrival: arrRateIn
    # ride back: backRateIn
    # ride break down: broRateIn
    # gathering: gathRateIn
    # fixing: fixRateIn
    # redistributing: redRateIn
    '''
    for f in [arrRateIn, backRateIn, broRateIn, gathRateIn, fixRateIn, redRateIn]:
        tempN, tempCol, tempVal = f(s)
        n += tempN
        col += tempCol
        val += tempVal
    
    return n, col, val

def generateR():
    #R = csr_matrix((S,S), dtype=np.float)
    Row, Col, Value = [], [], []
    for k, s in enumerate(State):
        '''
        number of row: n
        row number: k
        column number: col
        value: data
        '''
        # 加1
        if k==n_state-1: # collect the last row as a test instance
            tempN, tempCol, tempVal = getRateIn(s)
            tempCol += [k]
            tempVal += [getRateOut(s)]
        else:            # generate the mtx
            # set rate out for state s
            Row += [k]
            Col += [k]
            Value += [getRateOut(s)]

            # set rate in for state s
            tempN, tempCol, tempVal = getRateIn(s)
            Row += [k] * tempN
            Col += tempCol
            Value += tempVal

    Row += [k] * n_state
    Col += list(range(n_state))
    Value += [1] * n_state
    R = csr_matrix((Value, (Row, Col)), dtype=np.float) #.toarray()
    testArr = csr_matrix((tempVal, ([0]*(tempN+1), tempCol)), dtype=np.float)
    return R, testArr
BalanceMtx, testArr = generateR()



In [13]:
print(np.linalg.matrix_rank(BalanceMtx.toarray()))
#np.linalg.det(BalanceMtx.toarray())
BalanceMtx.toarray().shape

1267


(1267, 1267)

In [21]:
b = np.array([0]*(n_state-1) + [1])

x = spsolve(BalanceMtx, b)
x

array([1.15508557e-02, 1.15508557e-02, 1.15508557e-02, ...,
       4.34010798e-06, 2.96702537e-06, 6.17222497e-06])

In [22]:
sum(x)

1.0000000000000002

In [23]:
BalanceMtx.toarray()[-1].dot(x), testArr.toarray().dot(x)

(1.0000000000000004, array([3.81461288e-18]))

In [41]:
portionState = {}
with open('test.csv', 'w') as fout:
    for k,s in enumerate(State):
        portionState[s] = x[k]
        [fout.write(str(temp)+',') for temp in s]
        fout.write('%s\n'%(x[k]))
        #print(s, x[k])
#portionState

In [42]:
normalBikes, brokenBikes, idle, BP, RC, DP = 0,0,0,0,0,0
for k,s in enumerate(portionState):
    por = portionState[s]
    normalBikes += sum(s[:-3]) * por
    brokenBikes += sum(s[-3:]) * por
    if s[-2] == 0: idle += por
    BP += s[-3] * por
    RC += s[-2] * por
    DP += s[-1] * por
# normalBikes, brokenBikes, idle, BP, RC, DP
BP, DP, idle, normalBikes, RC

(1.4474438575351734,
 1.2916038786667852,
 0.5030781211581118,
 2.4631561086804084,
 0.7977961551176286)

In [35]:
names = ['c'+str(i) for i in range(n_queue)] + ['p']
df = pd.read_csv('test.csv', names=names)
df.head()

,c0,c1,c2,c3,c4,c5,c6,c7,c8,p
0,0,0,0,0,0,0,0,0,4,0.018907
1,0,0,0,0,0,0,0,1,3,0.009454
2,0,0,0,0,0,0,0,2,2,0.007090
3,0,0,0,0,0,0,0,3,1,0.003216
4,0,0,0,0,0,0,0,4,0,0.000973


In [17]:
# %store -d State
# %store -d portionState
cenState = State
cenPorState = portionState
%store cenState
%store cenPorState

Stored 'cenState' (dict)
Stored 'cenPorState' (dict)


In [46]:
portionState[(1,1,1,1,1,1,0,1,1)]

0.00016257952247527563

In [50]:
sorted(zip(portionState.values(), portionState.keys()), reverse=True)[5000]

(1.5573405349930055e-05, (3, 0, 0, 0, 0, 0, 3, 2, 0))